In [42]:
from pymongo import MongoClient

def insert_data():
    client = MongoClient('mongodb://localhost:27017/')
    db = client.local  # 'local' 데이터베이스 사용

    # 책 데이터 삽입
    books = [
        {"title": "Kafka on the Shore", "author": "Haruki Murakami", "year": 2002},
        {"title": "Norwegian Wood", "author": "Haruki Murakami", "year": 1987},
        {"title": "1Q84", "author": "Haruki Murakami", "year": 2009}
    ]
    db.books.insert_many(books)

    # 영화 데이터 삽입
    movies = [
        {"title": "Inception", "director": "Christopher Nolan", "year": 2010, "rating": 8.8},
        {"title": "Interstellar", "director": "Christopher Nolan", "year": 2014, "rating": 8.6},
        {"title": "The Dark Knight", "director": "Christopher Nolan", "year": 2008, "rating": 9.0}
    ]
    db.movies.insert_many(movies)

    # 사용자 행동 데이터 삽입
    user_actions = [
        {"user_id": 1, "action": "click", "timestamp": "2023-04-12T08:00:00Z"},
        {"user_id": 1, "action": "view", "timestamp": "2023-04-12T09:00:00Z"},
        {"user_id": 2, "action": "purchase", "timestamp": "2023-04-12T10:00:00Z"}
    ]
    db.user_actions.insert_many(user_actions)

    print("Data inserted successfully")
    client.close()

if __name__ == "__main__":
    insert_data()


Data inserted successfully


In [54]:
from pymongo import MongoClient
client =MongoClient('mongodb://localhost:27017/')
db = client.local
books_collection = db.books

books = [
        {"title": "Kafka on the Shore", "author": "Haruki Murakami", "year": 2002},
        {"title": "Norwegian Wood", "author": "Haruki Murakami", "year": 1987},
        {"title": "1Q84", "author": "Haruki Murakami", "year": 2009}
    ]
db.books.insert_many(books)


def find_fantasy_books(db,genre):
    books_collection =db.books
    query = {"genre": genre}
    projection ={"_id":0,"title":1,"author":1}

    books = books_collection.find(query,projection)
    for book in books:
        print(book)

find_fantasy_books(db,"fantasy")

In [55]:
clinet = MongoClient('mongodb://localhost:27017/')
db = client.local
movies_collection = db.movies
    
def rating_avg(db):
    movies_collection = db.movies
    pipeline =[ {"$group":{"_id":"$director","avgrating":{"$avg":"$rating"}}},
{"$sort":{"avgrating":-1}}
    ]
    results = movies_collection.aggregate(pipeline)
    for result in results:
        print(result)
rating_avg(db)

{'_id': 'Christopher Nolan', 'avgrating': 8.799999999999999}


In [56]:
actions_collection = db.user_actions
def get_recent_actions(user_id,limit=5):
    pipeline =[
        {"$match":{"user_id": user_id}},
        {"$sort":{"timestamp":-1}},
        {"$limit": limit}
    ]
    results =actions_collection.aggregate(pipeline)
    for result in results:
        print(result)

get_recent_actions(1)

{'_id': ObjectId('6821eea71d123b32c164e8dc'), 'user_id': 1, 'action': 'seen', 'timestamp': datetime.datetime(2023, 4, 9, 9, 0)}
{'_id': ObjectId('6821eea71d123b32c164e8db'), 'user_id': 1, 'action': 'click', 'timestamp': datetime.datetime(2023, 4, 8, 8, 0)}


In [57]:
books = [
        {"title": "Kafka on the Shore", "author": "Haruki Murakami", "year": 2002},
        {"title": "Norwegian Wood", "author": "Haruki Murakami", "year": 1987},
        {"title": "1Q84", "author": "Haruki Murakami", "year": 2009}
    ]
db.books.insert_many(books)


def get_books_by_year(db):
    books_collection = db.books
    pipeline = [ {"$group":{
        "_id" : "$year",
        "book_count":{"$sum":1}
    }},
    {"$sort":{"book_count":-1}}
    ]
    results = books_collection.aggregate(pipeline)
    for result in results:
        print(result)

get_books_by_year(db)

{'_id': 1987, 'book_count': 16}
{'_id': 2009, 'book_count': 16}
{'_id': 2002, 'book_count': 16}


In [58]:
from pymongo import MongoClient
from datetime import datetime


client = MongoClient('mongodb://localhost:27017/')
db = client.local


db.user_actions.delete_many({})


user_actions = [
    {"user_id": 1, "action": "click", "timestamp": datetime(2023, 4, 8, 8, 0)},
    {"user_id": 1, "action": "view",  "timestamp": datetime(2023, 4, 9, 9, 0)},
    {"user_id": 2, "action": "purchase", "timestamp": datetime(2023, 4, 12, 10, 0)}
]
db.user_actions.insert_many(user_actions)


def update_user_actions(db, user_id, before_date, old_action, new_action):
    user_actions = db.user_actions
    query = {
        "user_id": user_id,
        "action": old_action,
        "timestamp": {"$lt": before_date}
    }
    update = {
        "$set": {"action": new_action}
    }
    result = user_actions.update_many(query, update)
    print(f"matched: {result.matched_count}, modified: {result.modified_count}")


update_user_actions(db, 1, datetime(2023, 4, 10), "view", "seen")


matched: 1, modified: 1
